# ONETEP Calculations with ASE

This notebook demonstrates how to set up and run ONETEP calculations using the Atomic Simulation Environment (ASE) in Python, following the ONETEP tutorial.

## Setup

First, import the required modules:

In [ ]:
from ase.build import molecule
from ase.calculators.onetep import Onetep

## Simple SiH4 Molecule

Let's start with a simple silane (SiH4) molecule. Build the molecule using ASE:

In [ ]:
silane = molecule('SiH4', vacuum=10)

Define the ONETEP calculator with basic settings:

In [ ]:
calc = Onetep(
    cutoff_energy=500,
    species_ngwf_radius={'Si': 6.0, 'H': 6.0}, 
    species_ngwf_number={'Si': 4, 'H': 1}
)

Attach the calculator to the silane molecule and calculate the energy:

In [ ]:
silane.calc = calc
energy = silane.get_potential_energy()
print(f'Total energy: {energy:.4f} eV')

## Convergence Tests 

### Cutoff Energy

Test convergence with respect to cutoff energy:

In [ ]:
cutoffs = [300, 400, 500, 600, 700, 800]

for cutoff in cutoffs:
    calc = Onetep(cutoff_energy=cutoff)
    silane.calc = calc
    energy = silane.get_potential_energy()
    print(f'Cutoff: {cutoff:4d} eV, Energy: {energy:.4f} eV')

### NGWF Radius 

Test convergence with respect to NGWF radius:

In [ ]:
radii = [6.0, 7.0, 8.0, 9.0, 10.0]

for radius in radii:
    calc = Onetep(
        cutoff_energy=500,
        species_ngwf_radius={'Si': radius, 'H': radius})
    silane.calc = calc 
    energy = silane.get_potential_energy()
    print(f'NGWF radius: {radius:4.1f} Bohr, Energy: {energy:.4f} eV')

## Crystalline Silicon

Let's set up a calculation for crystalline silicon.

In [ ]:
from ase.build import bulk

# 2x2x2 supercell of 8-atom cubic unit cell
silicon = bulk('Si', 'diamond', a=10.1667, cubic=True).repeat(2)

Define ONETEP calculator with appropriate settings:

In [ ]:
calc = Onetep(
    cutoff_energy=600,
    species_ngwf_radius={'Si': 8.0},
    species_ngwf_number={'Si': 9},
    maxit_ngwf_cg=30, 
    ngwf_cg_max_step=8.0,
    write_forces=True
)

silicon.calc = calc
energy = silicon.get_potential_energy()
forces = silicon.get_forces()

print(f'Total energy: {energy:.4f} eV')
print(f'Forces:\n{forces}')